# Notebook to demonstrate visualizing external metabolite exchanges and reaction fluxes between multiple organisms

## Load modules

In [1]:
import cobra
import pandas as pd
import numpy as np
from concerto.visualizations import viz_utils
import plotly.graph_objects as go
import plotly.express as px

## Load Organism Models

In [2]:
from concerto.helpers.load_model_from_git import load_model_from_git

av = load_model_from_git('Azotobacter')
syn = load_model_from_git('Synechococcus')
rt = load_model_from_git('Rhodosporidium')

# Override synechococcus model since the sucrose
# will be removed soon and can utilize above
# syn = cobra.io.read_sbml_model('../models/synechococcus_elongatus_pcc_7942/iJB785_w_sucrose_transport.xml')

Set parameter Username
Set parameter GURO_PAR_SPECIAL
Set parameter TokenServer to value "leghorn"


### Update model ids: 
The "Id_Id" format prevents erasing org name entirely for each organism node in visualization

In [3]:
# format ==> "Id_Id"


# will this be needed without d3flux?

syn.id = "Se_Se"
rt.id = "Rt_Rt"
av.id = "Av_Av"
# Order matters!!! Should match order of row names in flux dataframe
org_ids = [av.id, rt.id, syn.id]

num_orgs = len(org_ids)

## Demonstrate default d3flux visualization for combined models

### Create organized d3flux map using the combined model and only shared external metabolites

In [24]:
# Create a model name used to label the new cobra model and save corresponding json files
combined_model_name = "AvRtSe_Metabolite_Exchange_Model_SharedOnly"

# Define figure boundaries and spacing between nodes for x & y directions
x_vals = {
    'min': -80,
    'max': 830,
    'space': 133
}
y_vals = {
    'min': 10,
    'max': 720,
    'space': 16
}

# Generate organism-metabolite map
fluxmap_dict = viz_utils.make_3organism_extmetab_viz(
    [av, syn, rt],
    model_name = combined_model_name,
    
    x_min = x_vals['min'],
    x_max = x_vals['max'],
    d_x = x_vals['space'],
    
    y_min = y_vals['min'],
    y_max = y_vals['max'],
    d_y = y_vals['space'],
)

# Display map
#display(fluxmap_dict["map"])

IndexError: list index out of range

### Create combined model for three organisms, now including all external metabolites

In [5]:
combined_model_name_all = "AvRtSe_Metabolite_Exchange_Model_All"

av_syn_rt_exchange_model_all_ext_m = viz_utils.make_combined_model_external_mets(
    [av, syn, rt],
    model_name = combined_model_name_all
)

In [6]:
av_syn_rt_exchange_model_all_ext_m

Name,AvRtSe_Metabolite_Exchange_Model_All
Memory address,0x025731ea28f0
Number of metabolites,489
Number of reactions,821
Number of groups,0
Objective expression,0.0
Compartments,"Av_Av, e, Se_Se, Rt_Rt"


In [7]:
# Define figure boundaries and spacing between nodes for x & y directions
x_vals = {
    'min': -20,
    'max': 830,
    'space': 130
}
y_vals = {
    'min': 10,
    'max': 730,
    'space': 15
}

# Generate organism-metabolite map
# NOTE: Multi-organism model passed in here to ensure the correct "all external metabolites" version is used.
#       Otherwise, a "shared metabolite only" version is created and used
fluxmap_dict = viz_utils.make_3organism_extmetab_viz(
    [av, syn, rt],
    model_name = combined_model_name_all,
    multi_org_model = av_syn_rt_exchange_model_all_ext_m,
    
    x_min = x_vals['min'],
    x_max = x_vals['max'],
    d_x = x_vals['space'],
    
    y_min = y_vals['min'],
    y_max = y_vals['max'],
    d_y = y_vals['space'],
)

# Display map
#display(fluxmap_dict["map"])

Showing map corresponding to the following file: AvRtSe_Metabolite_Exchange_Model_All_Updated.json


## Create Sankey/Alluvial diagrams to track external metabolimic flow between organisms

### Create combined model for three organisms, only including shared external metabolites

In [8]:
# Create a model name used to label the new cobra model and save corresponding json files
combined_model_name = "AvRtSe_Metabolite_Exchange_Model_SharedOnly"

av_syn_rt_exchange_model = viz_utils.make_combined_model_external_mets_shared_only(
    [av, syn, rt],
    model_name = combined_model_name
)

In [9]:
av_syn_rt_exchange_model

Name,AvRtSe_Metabolite_Exchange_Model_SharedOnly
Memory address,0x025767657520
Number of metabolites,196
Number of reactions,528
Number of groups,0
Objective expression,0.0
Compartments,"Av_Av, e, Se_Se, Rt_Rt"


### Import fluxes

In [25]:
org_ids

['Av_Av', 'Rt_Rt', 'Se_Se']

In [29]:
# Read in and manipulate df for flux info
# flux_fname = "../fluxes/Unedited_Medium_Fluxes.csv"
# flux_fname = "../fluxes/ternary_consortium_fluxes_w_glc_and_sucr_and_n2_no_leu_no_nh4.csv"

flux_fname = "../fluxes/test.csv"
# flux_fname = "../fluxes/ternary_model_co2_n2_growth_reduced_photons.csv"

flux_df = pd.read_csv(flux_fname)

# Create dict to match organism id's to df index/rownames
orgname_col = 'compartment'


# need more robust way. Depends on ordering of orgs. Should be able to use compartment name alone.
# pd.DataFrame().rename(dict()) would be agnostic to ordering
org_comp2id_dict = {flux_df[orgname_col][i]:org_ids[i] for i in range(num_orgs)}
org_comp2id_dict[flux_df[orgname_col][num_orgs]] = flux_df[orgname_col][num_orgs] # medium, currently not used but needed to match nrow(df)

# Define the organism IDs as the df index
flux_df.index = [org_comp2id_dict[k] for k in org_comp2id_dict]

display(flux_df.head())

,compartment,10FTHFGLULL,10FTHFGLULLm,12DGR120tipp,12DGR140tipp,12DGR141tipp,12DGR160tipp,12DGR161tipp,12DGR180tipp,12DGR181tipp,...,ZNabcpp,ZYMSTATer_RT,ZYMSTESTH_RT,ZYMSTESTtrd,ZYMSTt,ZYMSTtr,Zn2t,Zn2tex,ilvg,r0647
Av_Av,Azotobacter,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000732,NaN,NaN,NaN,NaN,NaN,NaN,0.000732,0.0,NaN
Rt_Rt,Rhodosporidium,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00367,0.0,0.000037,-0.291286,-0.291286,0.001101,NaN,NaN,0.0
Se_Se,medium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Synechococcus,Synechococcus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0


## Organize node and link info for Sankey/Alluvial diagram

### Define x-positions for different nodes

In [30]:
x_pos = {
    "source_m": 0.001,    # source metabolites that go into the system
    "donor_org": 0.25,    # organisms acting as donors
    "shared_m": 0.5,      # metabolites shared between more than two organisms
    "receiver_org": 0.75, # organisms acting as receivers
    "excreted_m": 0.999,  # output metabolites that go into the system
}

### Organize subclasses for metabolites

In [31]:
carrier_m = {
    'C': {
        'names': [],
        'color': 'rgb(78, 173, 91)' # Hex = 4EAD5B #'green'
    },
    'N': {
        'names': [],
        'color': 'rgb(202, 123, 44)' # Hex = CA7B2C # 'orange', #
    },
    'C+N': {
        'names': [],
        'color': 'rgb(0,0,1)' # 'blue',
    },
    'sunlight': {
        'names': [],
        'color': 'rgb(245, 194, 66)' # Hex = F5C242 # 'yellow',# 
    },
    'jet_fuel': {
        'list_of_m': [], # NEEDS TO BE FILLED WITH JET-FUEL CANDIDATE METABOLITES
        'names': [],
        'color': 'rgb(133, 26, 43)' # Hex = 851A2B # 'red', #
    },
}

### Add organisms into their respective carrier groups

In [32]:
# Synecococcus is the carbon carrier
carrier_m['C']['names'].append('Se_Se')

# Azotobacter is the nitrogen carrier
carrier_m['N']['names'].append('Av_Av')

# Rhodosporidium is the jet fuel creator
carrier_m['jet_fuel']['names'].append('Rt_Rt')

### Add corresponding metabolites into their respective carrier groups

In [33]:
for m in av_syn_rt_exchange_model_all_ext_m.metabolites:
    if m.id.lower().startswith("photon"):
        carrier_m['sunlight']['names'].append(m.id[:-2])
    elif ('C' in m.elements) and ('N' not in m.elements):
        carrier_m['C']['names'].append(m.id[:-2])
    elif ('C' not in m.elements) and ('N' in m.elements):
        carrier_m['N']['names'].append(m.id[:-2])
    elif ('C' in m.elements) and ('N' in m.elements):
        carrier_m['C+N']['names'].append(m.id[:-2])
    elif m.id in carrier_m['jet_fuel']['list_of_m']:
        carrier_m['jet_fuel']['names'].append(m.id[:-2])

carrier_names = []
for k in carrier_m:
    carrier_names.extend(carrier_m[k]['names'])

### Initialize dataframe for node information

In [34]:
# Construct node df with info about node index, class/type, and coordinates

# Add Donor organisms to node df
doner_df = pd.DataFrame.from_dict({
    "Node_Idx": np.arange(0, num_orgs),
    "Node_Name": org_ids,
    "Type": ["donor_org"]*num_orgs,
    "Xpos": [x_pos['donor_org']]*num_orgs,
    "Ypos": [np.nan]*num_orgs, #np.linspace(0.01, 0.99, num=Num_orgs) if Num_orgs>=3 else [0.33, 0.66] if Num_orgs==2 else [0.5],
    "Color": ['rgb(0,0,0)']*num_orgs
})

# Add Receiver organisms to node df
receiver_df = pd.DataFrame.from_dict({
    "Node_Idx": np.arange(num_orgs,(2*num_orgs)),
    "Node_Name": org_ids,
    "Type": ["receiver_org"]*num_orgs,
    "Xpos": [x_pos['receiver_org']]*num_orgs,
    "Ypos": [np.nan]*num_orgs, #np.linspace(0.01, 0.99, num=Num_orgs) if Num_orgs>=3 else [0.33, 0.66] if Num_orgs==2 else [0.5],
    "Color": ['rgb(0,0,0)']*num_orgs
})


# Add external metabolites to node df
# Note: needs refinement after considering reactions (see next cell)
ext_ms = [m for m in av_syn_rt_exchange_model_all_ext_m.metabolites if m.id not in org_ids]
ext_m_ids = [m.id[:-2] for m in ext_ms]
ext_m_ids.sort()
num_ms = len(ext_m_ids)

external_df = pd.DataFrame.from_dict({
    "Node_Idx": ((2*num_orgs)+np.arange(num_ms)),
    "Node_Name": ext_m_ids,
    "Type": ["metabolites"]*num_ms, # Will need updating
    "Xpos": [np.nan]*num_ms, # Will need updating
    "Ypos": [np.nan]*num_ms, # Will need updating,
    "Color": ['rgb(0,0,0)']*num_ms
})



node_df = pd.concat([doner_df, receiver_df, external_df], ignore_index=True)


### Generate links between nodes & refine x-positions of nodes

In [35]:
flux_threshold = 0.001

# Construct link df - first pass at connecting organisms to metabolites
links_df = pd.DataFrame(columns = ["Sources", "Source_Name","Targets", "Target_Name", "Values"])

m_in_use = []
# I assume the r.id[:-6] are from removing orgID_ordID?
# these indexes and namings can be improved.
for r in av_syn_rt_exchange_model_all_ext_m.reactions:
    if r.id[:-6] in flux_df.columns:
        # Recall that each reaction is between a single metabolite and a single organism
        this_m = r.id[3:-8]
        this_met_idx = node_df[(node_df['Node_Name']==this_m)]["Node_Idx"].values[0]
        this_org = r.id[-5:]
        this_val = flux_df.loc[this_org, r.id[:-6]]
        this_col = flux_df.loc[org_ids, r.id[:-6]]
        
        # Count good fluxes and flow direction
        sum_good_flux = num_orgs
        sum_pos_flux = 0
        sum_neg_flux = 0
        for v in this_col:
            if np.isnan(v) or abs(v)<=flux_threshold:
                sum_good_flux -= 1
            else:
                if abs(v)>flux_threshold and v>0:
                    sum_pos_flux += 1
                if abs(v)>flux_threshold and v<0:
                    sum_neg_flux += 1
        
        # Organize info for good reactions
        if (sum_good_flux >= 1) and (not np.isnan(this_val)) and (abs(this_val)>flux_threshold): 
            # Include metabolite into tracked list
            m_in_use.append(this_m)
            
            # Reaction from source-met to donor-org: overall, only negative fluxes
            if (sum_pos_flux==0) and (sum_neg_flux>0) and (this_val<0): # negative means uptake from the environment into the organism
                this_org_idx = node_df[(node_df["Node_Name"]==this_org) & (node_df["Type"]=="donor_org")]["Node_Idx"].values[0]
                links_df.loc[len(links_df.index)] = [this_met_idx,
                                                     this_m,
                                                     this_org_idx, 
                                                     this_org,
                                                     abs(this_val)]
                if np.isnan(node_df.loc[this_met_idx, "Xpos"]):
                    node_df.loc[this_met_idx, "Xpos"] = x_pos['source_m']
                    node_df.loc[this_met_idx, "Type"] = 'source_m'            

            # Reaction between shared-met and an org
            if (sum_pos_flux>0) and (sum_neg_flux>0):
                # Reaction from donor-org to shared-met:
                if this_val>0: # positive means secretion from the organism into the environment
                    this_org_idx = node_df[(node_df["Node_Name"]==this_org) & (node_df["Type"]=="donor_org")]["Node_Idx"].values[0]
                    links_df.loc[len(links_df.index)] = [this_org_idx,
                                                         this_org,
                                                         this_met_idx,
                                                         this_m,
                                                         abs(this_val)]
                
                # Reaction from shared-met to receiver-org:
                if this_val<0: # negative means uptake from the environment into the organism
                    this_org_idx = node_df[(node_df["Node_Name"]==this_org) & (node_df["Type"]=="receiver_org")]["Node_Idx"].values[0]
                    links_df.loc[len(links_df.index)] = [this_met_idx,
                                                         this_m,
                                                         this_org_idx,
                                                         this_org,
                                                         abs(this_val)]
                    
                if np.isnan(node_df.loc[this_met_idx, "Xpos"]):
                    if sum_pos_flux>sum_neg_flux: # More excreted than sourced
                        node_df.loc[this_met_idx, "Xpos"] = x_pos['shared_m'] - 0.15
                    elif sum_pos_flux==sum_neg_flux: # Equally excreted and sourced
                        node_df.loc[this_met_idx, "Xpos"] = x_pos['shared_m']
                    elif sum_pos_flux<sum_neg_flux: # More sourced than excreted
                        node_df.loc[this_met_idx, "Xpos"] = x_pos['shared_m'] + 0.15
                    node_df.loc[this_met_idx, "Type"] = 'shared_m'
            

            # Reaction from receiver-org to excreted-met: overall, only negative fluxes
            if (sum_pos_flux>0) and (sum_neg_flux==0) and (this_val>0): # positive means secretion from the organism into the environment
                this_org_idx = node_df[(node_df["Node_Name"]==this_org) & (node_df["Type"]=="receiver_org")]["Node_Idx"].values[0]
                links_df.loc[len(links_df.index)] = [this_org_idx,
                                                     this_org,
                                                     this_met_idx,
                                                     this_m,
                                                     abs(this_val)]
                if np.isnan(node_df.loc[this_met_idx, "Xpos"]):
                    node_df.loc[this_met_idx, "Xpos"] = x_pos['excreted_m']
                    node_df.loc[this_met_idx,"Type"] = 'excreted_m'

m_in_use = np.unique(m_in_use)


### Assign colors to nodes

In [36]:

# Color palette in hexadecimal format
myPalette = np.array(px.colors.qualitative.Light24) #np.array(px.colors.qualitative.Alphabet + px.colors.qualitative.Dark24 + px.colors.qualitative.Light24)

# Color palette as RGB tuples
myPalette_rgb = [tuple(int(c.lstrip('#')[i:i+2], 16) for i in (0, 2, 4)) for c in myPalette]
myPalette_rgb_list = [f'rgb{c}' for c in myPalette_rgb]

# Add colors to node_df
color_count = 0
org_idx = [i for i,n in enumerate(node_df['Node_Name']) if (n in org_ids)]
m_in_use_idx = [i for i,n in enumerate(node_df['Node_Name']) if (n in m_in_use)]

for i in org_idx+m_in_use_idx:
    this_node = node_df.loc[i, 'Node_Name']
    if this_node in carrier_names:
        for this_type in carrier_m:
            if this_node in carrier_m[this_type]['names']:
                node_df.loc[i, 'Color'] = carrier_m[this_type]['color']
    else:
        node_df.loc[i, 'Color'] = myPalette_rgb_list[color_count]
        color_count += 1
        if color_count==len(myPalette_rgb_list):
            color_count=0


### Assign y-positions to nodes based on color & x-position groupings

In [37]:
# Carrier type ordering
c_type_order = ['sunlight', 'C', 'N', 'jet_fuel']

for this_n_type in x_pos.keys():
    this_df = node_df[node_df['Type']==this_n_type]
    this_n_list = set(this_df['Node_Name'])
    this_yPos_N = len(this_n_list)
    new_n_list = []
    # Collect node-names in preferred ordered list
    for this_c_type in c_type_order:
        names_in_this_c_type = [n for n in this_df['Node_Name'] if n in carrier_m[this_c_type]['names']]
        if len(names_in_this_c_type)>0:
            # Include these names in list
            new_n_list.extend(names_in_this_c_type)
            
            # Add a spacer after each group
            this_yPos_N += 1
            new_n_list.append('space')
            # print(this_n_type, this_n_list, names_in_this_c_type)
            # Remove included nodes from this_n_list
            this_n_list.difference_update(names_in_this_c_type)
    # Collect remaining node-names into new_n_list
    new_n_list.extend(this_n_list)
    
    new_n_df = pd.DataFrame.from_dict({'Name': new_n_list, 'Ypos': np.linspace(0.01,0.99,this_yPos_N)})
    
    # Update node_df with y-values
    for i,n in enumerate(new_n_df['Name']):
        if not(n=='space'):
            node_df.loc[node_df['Node_Name']==n, 'Ypos'] = new_n_df.loc[i, 'Ypos']

In [38]:
node_df.head()

,Node_Idx,Node_Name,Type,Xpos,Ypos,Color
0,0,Av_Av,donor_org,0.25,0.402,"rgb(202, 123, 44)"
1,1,Rt_Rt,donor_org,0.25,0.794,"rgb(133, 26, 43)"
2,2,Se_Se,donor_org,0.25,0.010,"rgb(78, 173, 91)"
3,3,Av_Av,receiver_org,0.75,0.402,"rgb(202, 123, 44)"
4,4,Rt_Rt,receiver_org,0.75,0.794,"rgb(133, 26, 43)"


In [39]:
# Refine node labels for organisms
for org in org_ids:
    match_idx = node_df['Node_Name']==org
    node_df.loc[match_idx, 'Node_Name'] = flux_df.loc[org, "compartment"]
    node_df.loc[match_idx, 'Ypos'] = np.nan
node_df

,Node_Idx,Node_Name,Type,Xpos,Ypos,Color
0,0,Azotobacter,donor_org,0.250,NaN,"rgb(202, 123, 44)"
1,1,Rhodosporidium,donor_org,0.250,NaN,"rgb(133, 26, 43)"
2,2,medium,donor_org,0.250,NaN,"rgb(78, 173, 91)"
3,3,Azotobacter,receiver_org,0.750,NaN,"rgb(202, 123, 44)"
4,4,Rhodosporidium,receiver_org,0.750,NaN,"rgb(133, 26, 43)"
...,...,...,...,...,...,...
487,487,xylt,metabolites,NaN,NaN,"rgb(0,0,0)"
488,488,xylu__D,metabolites,NaN,NaN,"rgb(0,0,0)"
489,489,xylu__L,metabolites,NaN,NaN,"rgb(0,0,0)"
490,490,zn2,source_m,0.001,0.794,"rgb(201, 251, 229)"


In [40]:
# Force photon flux values to 1
for sn in links_df['Source_Name']:
    if 'photon' in sn:
        links_df.loc[links_df['Source_Name']==sn, 'Values'] = 1

In [41]:
flux_plot_filename = "Alluvial_Diagram_Ternary_CO2_N2_Growth_NoScale.html"

# Generate diagram
fig = go.Figure(go.Sankey(
    valueformat = ".3f",
    node=dict(
        pad = 15,
        thickness = 30,
        label=node_df['Node_Name'].values,
        x=list(node_df['Xpos'].values),
        y=list(node_df['Ypos'].values),
        color=node_df['Color'].values
            
    ),
    link=dict(
        arrowlen=30,
        source=links_df["Sources"].values,
        target=links_df["Targets"].values,
        value=links_df["Values"].values
    )
))

fig.update_layout(width=1200, height=700, margin={'t':10,'b':10})
fig.update_layout(font_size=16)
# fig.update_layout(title_text="Ternary Model Flux")
# fig.update_yaxes(autorange=True)
# fig.update_yaxes(autoshift=True)
fig.write_html(flux_plot_filename)
fig.show()